# Capstone Project - Week 5

# Table of Contents
[<li>Introduction</li>](#introduction)
[<li>Data</li>](#data)
[<li>Methodology</li>](#methodology)
[<li>Analysis</li>](#analysis)
[<li>Results and Discussion</li>](#results)
[<li>Conclusion</li>](#conclusion)

## Introduction<a name="introduction"></a>

"Pardes India Inc" is a large Indian restaurant chain operating out of MidWest USA. They already have few restaurants in Chicago metropolitan area and are looking to open Indian restaurants in many Chicago Suburbs. One of the key area they want to explore is Northwest suburb of Chicago. This area has sizeable Indian population and they believe opening an Indian restaurant there is a great business opportunity. However, they want identify a right location which is not crowded with Indian restaurants.

Schaumburg IL, is a suburb in the Northwest of Chicago. It is a multi-cultural suburb, famous for Woodfield Mall</b> and has offices of many multi-national corportions</b>. In this project, I will try to find suitable locations to open a new Indian restraurant.

Schaumburg has many zip codes, which have a large number of restaurants of different cusines. Apart from Schaumburg, I will also explore few other zip codes which are in close vicinity. Overall, I will explore ten (10) zip codes in and around Schaumburg and try to find locations that could be candidates for opening a new Indian restaurant.

## Data<a name="data"></a>

For this project, we will use different sources of data. Some of this data will be obtained by scraping various  websites and some of it will be fetched using Foursquare API. 

First, we will identify the suburbs in the vicinity of Schaumburg. Next we will get all the zip codes in these suburbs. Then, we will get some demographic details about these zipcodes and finally we will get the restaurant details in the identified zip codes. While collecting this information, we will also get location coordinates of suburbs, zipcodes as well as of the restaurants. 

We will use following sources to get the needed data.
1. Scrape Zipmat.net to get the zip codes for areas in vicinity of Schaumburg, IL.
2. Scrape zipdatamaps.com to get the Asian Population percentage, Total Population and Median Household income of each Zip Code. 
3. Geopy "geocode.Geolake" to get the coordinates for suburb and each zip code.
4. Use FourSquare API's to get the list of all restaurents and identify whether a restaurant is India or not.

## Methodology<a name="methodology"></a>

In this project, we will identify the zip codes in viciity of Schaumburg, that have low density of Indian restaurants. We will also get demographic details of each zip code to ensure we have enough Indian/Asian population in the zip code. Given that Indian cusine is typically little costlier than other cusines, we will also check the median Household Income in each zip code. and and the community.

In the first step, we will get the location details of all the suburbs we want to explore. And, with each suburb we will get the details of all zip codes in the suburb. In case, zip code is part of more than one suburb, we will assign it to the suburb which is nearest to it. We wil also get the population of the zipcode, percentage of Asian population as well as the Median Household income.

In the second step, we will do analysis to find the density of all restaurants as well as density of Indian restuarants in each zip code. We will use maps (including Heatmaps) to identify few promosing areas.

Finally in the third step, we will identify zip codes that have less density of Indian restuarants (less than 5%), where household Income is greater than 70K and Asian Population is at least 5% of the total population. We will cluster these locations to identity areas from where we can do further anaysis to pinpoint exact location for Indian Restaurant in discussion with the stakeholders.

## Analysis<a name="analysis"></a>

### Areas to explore

First we will list the suburbs in vicinity of Schaumburg, IL. For these suburbs, we will get the latitude and longitude details. Next, we will get zip codes in those suburbs. We will also get some demographic information for these zip codes. Finally, we will get list of restaurants in those zip codes and identify which ones are Indian restaurants.

Import all python libraries

In [1]:
#import the libraries.
import pandas as pd
from geopy.geocoders import Geolake
from geopy.geocoders import Nominatim
import geopy.distance
from bs4 import BeautifulSoup
import requests
import folium
from folium import plugins
from folium.plugins import HeatMap
from geopy.extra.rate_limiter import RateLimiter
from sklearn.cluster import KMeans

Create an list of suburbs in vicinity of Schaumburg that we want to explore for this project.

In [2]:
#Suburbs to explore in vicinity of Schaumburg
suburbs = ['Schaumburg, IL', 'Hoffman Estates, IL', 'Streamwood, IL',
           'Roselle, IL', 'Elk Grove Village, IL', 'Rolling Meadows, IL', 'Palatine, IL']

Create geolocator and set the links for website we will scrape to get zip codes and demographic data.

In [3]:
#Geolake API Key
api_key = 'hc3bNjfFhLC3oBi2cyLB'

#Create Geolocator instance
geolocator = Geolake(api_key,user_agent="city_explorer")
geolocator1 = Nominatim(user_agent="city_explorer1")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

#Link to websites we want to scrape to get zio code and demographic data.
weblink = "https://www.zipmap.net/Illinois/Cook_County/"
weblink1 = "https://www.zipdatamaps.com/"

#lists to hold Suburb details and ZipCodes around those suburbs
suburbAddress = []
zipCodes = []

Let's create few functions that we will use to get the suburb and zip code data

In [4]:
#Function to load suburbs and zipcodes to dataframes
def loadZipDF(suburbAddress,zip_Codes):
    suburbAddressDF = pd.DataFrame(suburbAddress)
    suburbAddressDF.columns = ['Suburb', 'Latitude', 'Longitude']

    zip_CodesDF = pd.DataFrame(zip_Codes)
    zip_CodesDF.columns = ['Zip Code', 'Suburb', 'Suburb Latitude', 'Suburb Longitude',
                           'Latitude', 'Longitude', 'Address', 'Distance from Suburb Center', 'Distance from Center',
                           'Asian Population', 'Total Population', 'Household Median Income']

    zip_CodesDF = zip_CodesDF[['Suburb', 'Suburb Latitude', 'Suburb Longitude', 'Zip Code', 
                               'Latitude', 'Longitude', 'Address', 'Distance from Suburb Center', 'Distance from Center',
                              'Asian Population', 'Total Population', 'Household Median Income']]    
    return suburbAddressDF, zip_CodesDF

#Function to get the cooridates.
def getCoordinates(item):
    location = geocode(item, timeout=10)
    return location

#Function to get demographic (Percent of Asian Population and Household Median Income)
def getDemographics(zipCode):

    url = weblink1 + zipCode
    req  = requests.get(url)
    data = req.text
    soup = BeautifulSoup(data,"lxml")

    #get percentage of Asians in the zip code
    tables = soup.find('table', class_="chart-legend table table-striped table-hover table-condensed")
    demoList1 = []   
    for row in tables.find_all(['tr']):
        column_marker = 0
        columns = row.find_all('td')
        tple = ()
        if len(columns) == 4:
            race = row.find('th')
            txt = race.text
            tple = (txt[1:],)
            for column in columns:
                
                if column_marker != 0:
                    txt1 = column.text 
                    txt1 = txt1[:-1]
                    tple = tple + (txt1,)
                    if column_marker == 1:
                        demoList1.append(list(tple))
                column_marker += 1
    asianPercent = [v for v in demoList1 if v[0] == 'Asian'] # Keep only Asian population
    asianPercent = [float(asianPercent[0][1])]

    #get the total population in the zip code    
    tables = soup.find_all('table', class_="table table-striped table-bordered table-hover table-condensed")
    demoList2 = []
    column_marker = 0
    for row in tables[1].find(['tr']):    
        if column_marker == 1:
            tple = row.text
            demoList2.append(tple)
        column_marker += 1
    population = [int(demoList2[0])]

    #get the Median Household Incomen in the zip code    
    demoList3 = []
    column_marker = 0
    for row in tables[3].find(['tr']):    
        if column_marker == 1:
            tple = row.text
            tple = tple[1:]
            demoList3.append(tple)
        column_marker += 1
    if tple == '':
        income = [0]
    else:
        income = [int(demoList3[0])]

    demographics = asianPercent + population + income
    
    return demographics

#Function to get the zip codes from the zipmat.net
def getZipCodes(suburb,latitude,longitude):
    suburb1 = suburb.split(',', 1)[0] #remove the state name from the suburb
    suburb1= suburb1.replace(' ', '_') #replace whitespaces with '_' to create proper url
    url = weblink + suburb1 + '.htm'
    point1 = (latitude, longitude)
    #read the url
    req  = requests.get(url) # get the data from url

    data = req.text
    soup = BeautifulSoup(data,"lxml") #parse the data
        
    lst = []
    for link in soup.find_all('a'): 
        lst.append(link.get('href'))

    # get the zip codes into a list
    for lst1 in lst:
        if'/zips/' in lst1:
            string = lst1.split('/zips/')[1].split(".htm")[0]
            location = getCoordinates(string)
            point2 = (location.latitude, location.longitude)
            distanceFromSuburb = geopy.distance.distance(point1, point2).m
            distanceFromCenter = geopy.distance.distance(latlng, point2).m
            demographics = getDemographics(string)
            zipCodes.append([string,suburb,latitude, longitude,location.latitude,location.longitude,
                             location.address,distanceFromSuburb,distanceFromCenter,
                             demographics[0],demographics[1],demographics[2]])
    return zipCodes

We will use zip code 60173 (Schaumburg) as the center of our search.

In [5]:
location = getCoordinates('60173') #Schaumburg zip code as city center
lat_ = location.latitude
lng_ = location.longitude
latlng = (lat_,lng_)
latlng

(42.0581, -88.0482)

Load the coordinates for Suburbs as well as the zip codes around them.

In [6]:
zipLoaded = False
#Try loading the Suburb and Zip Code details if we already have them from the previous run of this program
try:
    suburbAddressDF = pd.read_csv('c:\\rajesh dhar\\SuburbAddress.csv', index_col = None)
    suburbAddress = suburbAddressDF.values.tolist()

    zip_CodesDF = pd.read_csv('c:\\rajesh dhar\\ZipCodes.csv', index_col = None)
    zip_Codes = zip_CodesDF.values.tolist()
    print('Suburbs and Zip Codes Loaded')
    zipLoaded = True    

except:
    pass

demographicList = []
#If this is first time run of the program and we already don't have suburb and zipcodes csv files
if zipLoaded == False:
    for suburb in suburbs:
        print("Getting details for Suburban Area : ", suburb) # Print name of Suburb that is being processed

        # Get the coordinates of the suburb
        location = getCoordinates(suburb)
        latitude = location.latitude
        longitude = location.longitude
        address = location.address

        suburbAddress.append([suburb,latitude,longitude]) # save the address, latitude and longitude
        zipCodes = getZipCodes(suburb,latitude,longitude)  # get zip code details around the suburb  

    zipCodes.sort(key = lambda x: x[7], reverse=True) # sort the zip codes based on their distance

    #We will store zip codes in a dictionary to get rid of overlapping zipcodes and just keep the ones which are closet to suburb.
    zipCode = {}    
    for l1 in zipCodes:
        zipCode[l1[0]] = l1[1:]

    #Store truncated zipcodes to a list
    zip_Codes = [] 
    for key, value in zipCode.items():
        value.insert(0,key)            
        zip_Codes.append(value)

    #load suburbs and zipcodes to dataframes
    suburbAddressDF, zip_CodesDF = loadZipDF(suburbAddress,zip_Codes) #,demographics)
#    suburbAddressDF = pd.merge(suburbAddressDF, demoDF, on=['Suburb'], how='inner')

    zip_CodesDF.sort_values(by=['Suburb','Zip Code'], inplace= True) # sort zip codes within suburb
    zip_CodesDF = zip_CodesDF[zip_CodesDF['Total Population'] != 0]

    #Save the suburbs and zipcodes to csv files for next excution of this progrm
    suburbAddressDF.to_csv('c:\\rajesh dhar\\SuburbAddress.csv', index=False)
    zip_CodesDF.to_csv('c:\\rajesh dhar\\ZipCodes.csv', index=False)

    print('Suburbs and Zip Codes Loaded')

Getting details for Suburban Area :  Schaumburg, IL
Getting details for Suburban Area :  Hoffman Estates, IL
Getting details for Suburban Area :  Streamwood, IL
Getting details for Suburban Area :  Roselle, IL
Getting details for Suburban Area :  Elk Grove Village, IL
Getting details for Suburban Area :  Rolling Meadows, IL
Getting details for Suburban Area :  Palatine, IL
Suburbs and Zip Codes Loaded


In [7]:
print('Coordinates for {} Suburbs and {} Zip Codes Loaded'.format(len(suburbAddressDF), len(zip_CodesDF)))

Coordinates for 7 Suburbs and 30 Zip Codes Loaded


In [8]:
#List of suburb with latitude, Longitude and Demographic details
suburbAddressDF

,Suburb,Latitude,Longitude
0,"Schaumburg, IL",42.029075,-88.090953
1,"Hoffman Estates, IL",42.065890,-88.121643
2,"Streamwood, IL",42.021546,-88.175984
3,"Roselle, IL",41.980013,-88.089189
4,"Elk Grove Village, IL",42.006304,-88.005855
5,"Rolling Meadows, IL",42.076532,-88.025688
6,"Palatine, IL",42.119485,-88.041359


In [9]:
map_schaumburg = folium.Map(location=[lat_, lng_], zoom_start=11)
for row in suburbAddress:
    latilongi = [row[1],row[2]]
    popup = row[0]
    folium.Marker(latilongi, popup=popup).add_to(map_schaumburg)
map_schaumburg

In [10]:
zip_CodesDF[5:15]

,Suburb,Suburb Latitude,Suburb Longitude,Zip Code,Latitude,Longitude,Address,Distance from Suburb Center,Distance from Center,Asian Population,Total Population,Household Median Income
32,"Elk Grove Village, IL",42.006304,-88.005855,60191,41.9602,-87.9810,"Wood Dale, US",5519.642413,12216.170165,4.41,14310,71567
12,"Hoffman Estates, IL",42.065890,-88.121643,60169,42.0493,-88.1065,"Hoffman Estates, US",2228.640339,4924.124896,23.59,33847,63780
18,"Hoffman Estates, IL",42.065890,-88.121643,60192,42.0428,-88.0798,"Hoffman Estates, US",4309.923768,3119.554152,28.29,16343,108651
19,"Hoffman Estates, IL",42.065890,-88.121643,60195,42.0764,-88.1093,"Schaumburg, US",1551.207998,5450.085925,38.03,4769,83358
33,"Palatine, IL",42.119485,-88.041359,60004,42.1120,-87.9792,"Arlington Heights, US",5207.375760,8272.669875,8.35,50582,83431
8,"Palatine, IL",42.119485,-88.041359,60010,42.1614,-88.1383,"Barrington, US",9268.193450,13681.636887,10.34,44095,133572
7,"Palatine, IL",42.119485,-88.041359,60047,42.2165,-88.0769,"Lake Zurich, US",11169.116825,17753.694608,8.55,41669,125778
9,"Palatine, IL",42.119485,-88.041359,60067,42.1139,-88.0429,"Palatine, US",633.311587,6213.475959,14.38,38585,90474
26,"Palatine, IL",42.119485,-88.041359,60074,42.1458,-88.0230,"Palatine, US",3293.585446,9961.831594,12.97,38985,72946
24,"Palatine, IL",42.119485,-88.041359,60089,42.1598,-87.9644,"Buffalo Grove, US",7780.084556,13253.194850,17.81,41533,101576


Let's count how many zip codes are in each suburb

In [11]:
zip_CodesDF.groupby(['Suburb'])['Zip Code'].count()

Suburb
Elk Grove Village, IL    6
Hoffman Estates, IL      3
Palatine, IL             6
Rolling Meadows, IL      5
Schaumburg, IL           4
Streamwood, IL           6
Name: Zip Code, dtype: int64

Okay each suburb has fairly similar number of zip codes around it

Let's plot the zip codes on the Map to see how are they scattered arond our center.

In [12]:
map_schaumburg = folium.Map(location=[lat_, lng_], zoom_start=11)
for row in suburbAddress:
    latilongi = [row[1],row[2]]
    popup = row[0]
    folium.Marker(latilongi, popup=popup).add_to(map_schaumburg)

for lat, lon, addr, zipcde in zip(zip_CodesDF['Latitude'], zip_CodesDF['Longitude'], zip_CodesDF['Address'], zip_CodesDF['Zip Code']):
    label = '{}, {}'.format(zipcde, addr)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=3, color='blue', fill=True, popup=label,
                        fill_color='blue', fill_opacity=1).add_to(map_schaumburg) 
map_schaumburg

Okay, we got Zip Codes for the Suburbs around Schaumburg. Zip Codes seem to be concentrated towards west, east and south of Schuamburng.

Now Lets get the deatils of restaurants around these zip codes and identify the Indian restaurants among them. We will do this using Foursquare API.

Define Foursquare Credentials and Version

In [13]:
client_id = '24ALNEKCXXGKZP1LR5I4RVXC5PLEBUSVI2H3N5NXHGSDKMFC' # your Foursquare ID
client_secret = '3YYXMPCUOHNXKQGM13QRWBHICR1Z1FEOG41RKXRGNUDU0N5M' # your Foursquare Secret
version = '20180605'

Let's use Foursquare category for 'food' to restrict the type of venues we get.

In [14]:
#Four Squarre id for Food
food_category = '4d4b7105d754a06374d81259'

Let's create few functions that will get us restaurant data. We will also store this data in csv files.

In [15]:
#Function to replace country name from the address
def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', United States', '')
    return address

In [16]:
#Function to get list of restaurants using Foursquare API
def getRestaurants(lat, lon, radius=500, limit=100):
   
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}\
        &v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, food_category, radius, limit)
            
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    venues = [(item['venue']['id'],
                   item['venue']['name'],
                   item['venue']['categories'][0]['name'],
                   item['venue']['location']['lat'], 
                   item['venue']['location']['lng'],
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        

    return venues

In [17]:
#Function to load restaurants to dataframes
def loadRestaurantDF(all_restaurants, indian_restaurants):
    all_restaurantsDF = pd.DataFrame(all_restaurants)
    all_restaurantsDF.columns = ['Venue Id', 'Venue Name', 'Venue Category', 'Venue Latitude',
                               'Venue Longitude', 'Venue Location', 'Venue Distance',
                               'Zip Code', 'Is Indian', 'Distance From Center']

    indian_restaurantsDF = pd.DataFrame(indian_restaurants)
    indian_restaurantsDF.columns = ['Venue Id', 'Venue Name', 'Venue Category', 'Venue Latitude',
                               'Venue Longitude', 'Venue Location', 'Venue Distance',
                               'Zip Code', 'Is Indian', 'Distance From Center']
    
    return all_restaurantsDF, indian_restaurantsDF

Let's get the restaurants aroud the Zip Codes within the radius of 3KM. We will load these restaurants in two csv files. First file for all restaurants and second file only for Indian restaurants. We will use these files in subsquent program executions to avoid using Foursquare API.

In [18]:
loaded = False
#Try loading the restaurants if we already have them from the previous run of this program
try:
    all_restaurantsDF = pd.read_csv('C:\\rajesh dhar\\all_restaurants.csv', index_col = None)
    allRestaurants = all_restaurantsDF.values.tolist()    
    
    indian_restaurantsDF = pd.read_csv('C:\\rajesh dhar\\indian_restaurants.csv', index_col = None)
    indianRestaurants = indian_restaurantsDF.values.tolist()    
    
    loaded = True
    print('Restaurant data Loaded')

except:
    pass

if not loaded:

    restaurants ={}
    indian_restaurants = {} 
    indian_restaurant = []

    all_restaurants = {}
    all_restaurant = []

    venues = []    
    radius = 3000 #3KM Define the radius for Foursquare API
    limit = 100
    for lati, longi, zipCode in zip(zip_CodesDF['Latitude'], zip_CodesDF['Longitude'],zip_CodesDF['Zip Code']):
        print('Fetching restaurants for Zip Code ', zipCode)
        venues = getRestaurants(lati, longi, radius,limit)

        for venue in venues:
            venueId = venue[0]
            venueName = venue[1]
            venueCat = venue[2]
            venueLat = venue[3]
            venueLon = venue[4]
            venueLoc = venue[5]
            venueDist = venue[6]

            point2 = (venueLat, venueLon)
            distanceFromCenter = geopy.distance.distance(latlng, point2).m #distance of venue from city center

            if 'Indian' in venueCat: #Check if restaurant category is Indian
                isIndian = True
            else:
                isIndian = False

            all_restaurant.append([venueId, venueName, venueCat, venueLat, venueLon, venueLoc, venueDist, 
                          zipCode, isIndian, distanceFromCenter])     
            if isIndian:
                indian_restaurant.append([venueId, venueName, venueCat, venueLat, venueLon, venueLoc, venueDist, 
                          zipCode, isIndian, distanceFromCenter])     

        all_restaurant.sort(key = lambda x: x[6], reverse=True)
        indian_restaurant.sort(key = lambda x: x[6], reverse=True)

        #store restaurants in a dictionary to get rid of overlapping restaurants and just keep the ones which are closet to zip code.        
        for l1 in all_restaurant:
            all_restaurants[l1[0]] = l1[1:]             
        for l2 in indian_restaurant:
            indian_restaurants[l2[0]] = l2[1:]

        allRestaurants = [] 
        indianRestaurants = [] 
        
        #Store truncated zipcodes to a list
        for key, value in all_restaurants.items():
            value.insert(0,key)            
            allRestaurants.append(value)
        for key, value in indian_restaurants.items():
            value.insert(0,key)            
            indianRestaurants.append(value)
          
    all_restaurantsDF, indian_restaurantsDF = loadRestaurantDF(allRestaurants, indianRestaurants)

    #sort the dataframes for restaurants name within Zip Codes
    all_restaurantsDF.sort_values(by=['Zip Code','Venue Name'], inplace=True)
    indian_restaurantsDF.sort_values(by=['Zip Code','Venue Name'], inplace=True)
    
    #Save the restaurants to csv files for next excution of this progrm
    all_restaurantsDF.to_csv('c:\\rajesh dhar\\all_restaurants.csv', index=False)
    indian_restaurantsDF.to_csv('c:\\rajesh dhar\\Indian_restaurants.csv', index=False)
    loaded = True
    print('Restaurant data Loaded')

Fetching restaurants for Zip Code  60007
Fetching restaurants for Zip Code  60101
Fetching restaurants for Zip Code  60106
Fetching restaurants for Zip Code  60143
Fetching restaurants for Zip Code  60157
Fetching restaurants for Zip Code  60191
Fetching restaurants for Zip Code  60169
Fetching restaurants for Zip Code  60192
Fetching restaurants for Zip Code  60195
Fetching restaurants for Zip Code  60004
Fetching restaurants for Zip Code  60010
Fetching restaurants for Zip Code  60047
Fetching restaurants for Zip Code  60067
Fetching restaurants for Zip Code  60074
Fetching restaurants for Zip Code  60089
Fetching restaurants for Zip Code  60005
Fetching restaurants for Zip Code  60008
Fetching restaurants for Zip Code  60056
Fetching restaurants for Zip Code  60070
Fetching restaurants for Zip Code  60173
Fetching restaurants for Zip Code  60108
Fetching restaurants for Zip Code  60172
Fetching restaurants for Zip Code  60193
Fetching restaurants for Zip Code  60194
Fetching restaur

In [19]:
print('Total number of restaurants:', len(allRestaurants))
print('Total number of Indian restaurants:', len(indianRestaurants))
print('Percentage of Indian restaurants: {:.2f}%'.format(len(indianRestaurants) / len(allRestaurants) * 100))
print('Average number of restaurants in Zip Code: {:.2f}'.format(len(allRestaurants)/len(all_restaurantsDF['Zip Code'].unique())))

Total number of restaurants: 1413
Total number of Indian restaurants: 32
Percentage of Indian restaurants: 2.26%
Average number of restaurants in Zip Code: 47.10


Let's plot restaurants and see where they fit on the Map. We will try to see where are the Indian restaurants in these zip codes.

In [20]:
map_schaumburg = folium.Map(location=[lat_, lng_], zoom_start=11)
for row in suburbAddress:
    latilongi = [row[1],row[2]]
    popup = row[0]
    folium.Marker(latilongi, popup=popup).add_to(map_schaumburg)
for lat, lon, name, isIndian in zip(all_restaurantsDF['Venue Latitude'], all_restaurantsDF['Venue Longitude'], all_restaurantsDF['Venue Name'],all_restaurantsDF['Is Indian']):
    color = 'red' if isIndian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True,
                        fill_color=color, fill_opacity=1).add_to(map_schaumburg)
map_schaumburg

Alright, We don't see lot of Indian restaurants (red circles). But as we zoom in, we do see there are areas where there is significant concentration of Indian restaurents.

In [21]:
#Print list of all restaurants
all_restaurantsDF[5:15]

,Venue Id,Venue Name,Venue Category,Venue Latitude,Venue Longitude,Venue Location,Venue Distance,Zip Code,Is Indian,Distance From Center
457,4bb3edbc87aa95218cb0acd1,China Wok,Asian Restaurant,42.109857,-87.965770,"1451 E Palatine Rd (at Windsor Dr), Arlington ...",1134,60004,False,8920.048974
475,4b58a8e8f964a520336428e3,Chipotle Mexican Grill,Mexican Restaurant,42.112781,-87.978320,338 E Rand Rd (Off Rand Rd. in the Northpoint ...,113,60004,False,8385.686547
468,4b5345dff964a520369527e3,Corner Bakery Cafe,Bakery,42.110841,-87.975805,"470 E Rand Rd,, Arlington Heights, IL 60004",308,60004,False,8378.491951
476,4be9e1346295c9b6b6c98508,Domino's Pizza,Pizza Place,42.111412,-87.979336,325 E Rand Rd (Rand Rd and Arlingtron Heights ...,66,60004,False,8217.796521
429,4c10dbd5ce57c928977382d2,Dunkin' Donuts,Donut Shop,42.138687,-87.984020,"105 W Dundee Rd, Arlington Heights, IL 60004",2997,60004,False,10407.311238
459,4a677de8f964a52088c91fe3,Dunkin' Donuts,Donut Shop,42.106210,-87.970440,"1010 E Rand Rd, Arlington Heights, IL 60004",968,60004,False,8363.864215
473,5830948a24ca6a7d84491b62,Gail's Carriage Place,Breakfast Spot,42.113484,-87.979423,"306 E Rand Rd, Arlington Heights, IL 60004",166,60004,False,8380.111858
474,4f74eb2ce4b07d85bfba0b52,Garibaldi's,Italian Restaurant,42.112069,-87.981041,"1960 N Arlington Heights Rd, Arlington Heights...",152,60004,False,8173.920213
466,5c2becf092e7a9002c478095,Golden Corral,Buffet,42.108451,-87.976266,"445 E Palatine Rd, Arlington Heights, IL 60004",463,60004,False,8167.316958
432,4c9002135fdf6dcb1e2a2d91,Gulo's Pizza Paddle,Pizza Place,42.138019,-87.985620,"1231 W Dundee Rd, Buffalo Grove, IL 60089",2944,60004,False,10276.208789


In [22]:
indian_restaurantsDF[5:15]

,Venue Id,Venue Name,Venue Category,Venue Latitude,Venue Longitude,Venue Location,Venue Distance,Zip Code,Is Indian,Distance From Center
26,57d5e407498efd6250f1f507,India Foodie,North Indian Restaurant,42.019324,-88.144909,"698 S Barrington Rd, Streamwood, IL 60107",2023,60107,True,9092.373020
0,4bd4cd02cfa7b7135ce724da,Cool Mirchi,Indian Restaurant,41.999869,-88.059911,"814 E Nerge Rd, Roselle, IL 60172",2178,60157,True,6540.290726
3,52b4efd7498e25b1f6f44607,Atithi,Indian Restaurant,42.048326,-88.084375,"167 W Golf Rd, Schaumburg, IL 60195",1832,60169,True,3185.343229
4,4e14fbada809f31f66a2d93c,Atithi,Indian Restaurant,42.048390,-88.084402,"167 W Golf Rd, Schaumburg, IL 60195",1829,60169,True,3185.027308
8,4bc9e55b68f976b01e115e83,Dakshin Indian Cuisine,Indian Restaurant,42.047734,-88.098834,"1135 N Salem Dr (at Golf Rd.), Schaumburg, IL ...",657,60169,True,4346.830056
5,4c040f57f56c2d7ffbbb1d66,Hakka Wok,Indian Restaurant,42.046031,-88.125197,"1851 W Golf Rd, Schaumburg, IL 60194",1587,60169,True,6513.475563
10,56f8784a498e69128da0003b,Honest,Indian Restaurant,42.049291,-88.101547,"835 W Higgins Rd, Schaumburg, IL 60195",409,60169,True,4523.187453
12,4c4c8be05609c9b6e945bc92,Hot Breads,Indian Restaurant,42.047799,-88.105356,"1065 W Golf Rd, Hoffman Estates, IL 60169",192,60169,True,4867.857289
6,55691b98498e033029e8b641,Inchin's Bamboo Garden,Indian Chinese Restaurant,42.046619,-88.124826,"Schaumburg, IL",1544,60169,True,6470.228066
7,4d8518c1d5fab60c1579f09b,India House Restaurant,Indian Restaurant,42.047927,-88.097757,"721 W Golf Rd (at Higgins Rd.), Hoffman Estate...",738,60169,True,4255.158053


We are done with the data collection for the project. We may need some additional data later, but for now we are good. We will use this data in Week 5 Lab to perform our analysis.

### Analyze the Data

Now let's us analyze the data.
1. Find how many restaurants are in each zip code and what is their minimum and average distane from center of zip code.
2. Find how many Indina restaurants are in each zip code and what is their minimum and average distane from center of zip code.
3. Calculate the percentage of Indian restaurants in each Zip Code
4. Plot the heatmap of all restaurants and Indian restaurants only.

Get count of all restaurants  in a zip code. 

In [23]:
count_of_all_restaurants = all_restaurantsDF.groupby('Zip Code')['Is Indian'].count()
min_distance_from_zipAll = all_restaurantsDF.groupby('Zip Code').min()['Venue Distance']
avg_distance_from_zipAll = all_restaurantsDF.groupby('Zip Code').mean()['Venue Distance']
zipAllDF = pd.DataFrame({'Zip Code':count_of_all_restaurants.index, 'Num of All Restaurants':count_of_all_restaurants.values,
                        'Min Distance from Zip':min_distance_from_zipAll.values,'Avg Distance from Zip':avg_distance_from_zipAll.values})
zipAllDF = zipAllDF[['Zip Code', 'Num of All Restaurants','Min Distance from Zip','Avg Distance from Zip']]
zipAllDF['Avg Distance from Zip'] = round(zipAllDF['Avg Distance from Zip'],0)
zipAllDF[5:20]

,Zip Code,Num of All Restaurants,Min Distance from Zip,Avg Distance from Zip
5,60047,11,2099,2366.0
6,60056,73,75,1781.0
7,60067,49,299,1362.0
8,60070,40,658,2081.0
9,60074,68,413,1638.0
10,60089,77,286,1810.0
11,60101,53,520,1634.0
12,60103,32,105,1340.0
13,60106,42,587,1671.0
14,60107,55,1098,2130.0


In [24]:
print('Average distance to closest restaurant from each Zip Code center:', zipAllDF['Min Distance from Zip'].mean())

Average distance to closest restaurant from each Zip Code center: 570.8


Get count of Indian restaurants  in a zip code. 

In [25]:
count_of_indian_restaurants = indian_restaurantsDF.groupby('Zip Code')['Is Indian'].count()
min_distance_from_zipIndian = indian_restaurantsDF.groupby('Zip Code').min()['Venue Distance']
avg_distance_from_zipIndian = indian_restaurantsDF.groupby('Zip Code').mean()['Venue Distance']
zipIndianDF = pd.DataFrame({'Zip Code':count_of_indian_restaurants.index, 'Num of Indian Restaurants':count_of_indian_restaurants.values,
                        'Min Distance from Zip-Indian':min_distance_from_zipIndian.values,'Avg Distance from Zip-Indian':avg_distance_from_zipIndian.values})
zipIndianDF = zipIndianDF[['Zip Code', 'Num of Indian Restaurants','Min Distance from Zip-Indian','Avg Distance from Zip-Indian']]
zipIndianDF['Avg Distance from Zip-Indian'] = round(zipIndianDF['Avg Distance from Zip-Indian'],0)

zipIndianDF

,Zip Code,Num of Indian Restaurants,Min Distance from Zip-Indian,Avg Distance from Zip-Indian
0,60056,1,1749,1749.0
1,60074,2,744,904.0
2,60089,2,362,1047.0
3,60107,1,2023,2023.0
4,60157,1,2178,2178.0
5,60169,12,91,1045.0
6,60173,2,1262,1742.0
7,60188,2,2989,2990.0
8,60192,3,708,1548.0
9,60193,3,1228,1649.0


In [26]:
print('Average distance to closest Indian restaurant from each Zip Code center:', zipIndianDF['Min Distance from Zip-Indian'].mean())

Average distance to closest Indian restaurant from each Zip Code center: 1368.4545454545455


Get the percentage of Indian restaurants in each Zip Code

In [27]:
zipMergedDF = zipAllDF.join(zipIndianDF.set_index('Zip Code'), on='Zip Code',how = 'outer').fillna(0)
zipMergedDF['Percent of Indian Restaurants'] = round(zipMergedDF['Num of Indian Restaurants']/zipMergedDF['Num of All Restaurants']*100,2)
zipMergedDF[(zipMergedDF['Percent of Indian Restaurants'] > 0)]

,Zip Code,Num of All Restaurants,Min Distance from Zip,Avg Distance from Zip,Num of Indian Restaurants,Min Distance from Zip-Indian,Avg Distance from Zip-Indian,Percent of Indian Restaurants
6,60056,73,75,1781.0,1.0,1749.0,1749.0,1.37
9,60074,68,413,1638.0,2.0,744.0,904.0,2.94
10,60089,77,286,1810.0,2.0,362.0,1047.0,2.60
14,60107,55,1098,2130.0,1.0,2023.0,2023.0,1.82
19,60157,41,560,2196.0,1.0,2178.0,2178.0,2.44
20,60169,49,88,1269.0,12.0,91.0,1045.0,24.49
22,60173,79,294,1396.0,2.0,1262.0,1742.0,2.53
24,60188,56,298,2235.0,2.0,2989.0,2990.0,3.57
26,60192,46,109,1107.0,3.0,708.0,1548.0,6.52
27,60193,51,1003,1496.0,3.0,1228.0,1649.0,5.88


We find that there are lot of Zip Codes with high restaurant density and very low density of Indian restaurants. Some zip codes have no Indian restaurants at all. Zip Codes 60169, 60194 have high density of Indian restaurants, while zip codes 60192 and 60193 have fair density of Indian restaurants.

Let's get the latitude and Longitude of all restauurants as well as India restaurants

In [28]:
#get the latitude and Longitude of all restauurants as well as India restaurants
dataAll = []
for allCoordinates in allRestaurants:
    dataAll.append(allCoordinates[3:5])
dataIndian = []
for indianCoordinates in indianRestaurants:
    dataIndian.append(indianCoordinates[3:5])

Let's plot the restaurants in a heat map of all restaurants

In [29]:
map_schaumburg = folium.Map(location=[lat_, lng_], zoom_start=11)
for row in suburbAddress:
    latilongi = [row[1],row[2]]
    popup = row[0]
    folium.Marker(latilongi, popup=popup).add_to(map_schaumburg)
HeatMap(dataAll).add_to(map_schaumburg)
folium.Circle(location=[lat_, lng_], radius=5000, fill=False, color='white').add_to(map_schaumburg)
folium.Circle(location=[lat_, lng_], radius=9000, fill=False, color='white').add_to(map_schaumburg)
folium.Circle(location=[lat_, lng_], radius=15000, fill=False, color='white').add_to(map_schaumburg)
map_schaumburg

Looks like there are few suburbs where there are not many restaurants.There aren't many restaurants in North east of Schaumburg.

Let's plot the restaurants in a heat map of Indian restaurants

In [30]:
map_schaumburg = folium.Map(location=[lat_, lng_], zoom_start=11)
for row in suburbAddress:
    latilongi = [row[1],row[2]]
    popup = row[0]
    folium.Marker(latilongi, popup=popup).add_to(map_schaumburg)
HeatMap(dataIndian).add_to(map_schaumburg)
folium.Circle(location=[lat_, lng_], radius=5000, fill=False, color='white').add_to(map_schaumburg)
folium.Circle(location=[lat_, lng_], radius=9000, fill=False, color='white').add_to(map_schaumburg)
folium.Circle(location=[lat_, lng_], radius=15000, fill=False, color='white').add_to(map_schaumburg)
map_schaumburg

There seem to be lot of Indian restaurants between trinagle of Schaumburg, Hoffman Estates and Streamwood

### Identify Zip Codes where Indian restaurants can be opened.

We will identify zip codes that meet following criteria:
1. Percentage of Indian restaurants is less than 3%
3. Asian population is more than 5% of total population
4. Household Median Income is above 70K

In [32]:
zipRestaurants = zip_CodesDF.merge(zipMergedDF,on='Zip Code',how='inner')

In [33]:
#zipRestaurants1 = zipRestaurants[zipRestaurants['Num of All Restaurants'] <= 100]

zipRestaurants1 = zipRestaurants[(zipRestaurants['Percent of Indian Restaurants'] <= 3) & 
                                 (zipRestaurants['Household Median Income'] >=70000) &
                                 (zipRestaurants['Asian Population'] >=5)]
zipRestaurants1[['Zip Code', 'Asian Population','Household Median Income','Percent of Indian Restaurants' ]]

,Zip Code,Asian Population,Household Median Income,Percent of Indian Restaurants
0,60007,10.59,75743,0.00
3,60143,7.44,88031,0.00
4,60157,5.05,77536,2.44
8,60195,38.03,83358,0.00
9,60004,8.35,83431,0.00
10,60010,10.34,133572,0.00
11,60047,8.55,125778,0.00
12,60067,14.38,90474,0.00
13,60074,12.97,72946,2.94
14,60089,17.81,101576,2.60


Let's plot these zip codes on the map to see they are locatated

In [34]:
map_schaumburg = folium.Map(location=[lat_, lng_], zoom_start=11)
for row in suburbAddress:
    latilongi = [row[1],row[2]]
    popup = row[0]
    folium.Marker(latilongi, popup=popup).add_to(map_schaumburg)
for lat, lon in zip(zipRestaurants1['Latitude'], zipRestaurants1['Longitude']):
    color = 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True,
                        fill_color=color, fill_opacity=1).add_to(map_schaumburg)
map_schaumburg

Now let's cluster these zip codes to create center of zip codes. These zip codes and their centers and addresses will be the final result of our analysis.

In [35]:
dataZipCodes = []
for allCoordinates in zipRestaurants1.values.tolist():
    dataZipCodes.append(allCoordinates[4:6])

In [36]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dataZipCodes)

In [37]:
# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([4, 4, 4, 0, 3, 2, 2, 0, 3, 3, 0, 3, 0, 4, 4, 1, 1, 1])

In [38]:
# check cluster centers generated for each row in the dataframe
kmeans.cluster_centers_

array([[ 42.08035   , -88.054875  ],
       [ 42.00046667, -88.17346667],
       [ 42.18895   , -88.1076    ],
       [ 42.12      , -87.976075  ],
       [ 41.97782   , -88.04568   ]])

In [39]:
# add clustering labels
zipRestaurants1.insert(0, 'Cluster Labels', kmeans.labels_)
zipRestaurants1

,Cluster Labels,Suburb,Suburb Latitude,Suburb Longitude,Zip Code,Latitude,Longitude,Address,Distance from Suburb Center,Distance from Center,Asian Population,Total Population,Household Median Income,Num of All Restaurants,Min Distance from Zip,Avg Distance from Zip,Num of Indian Restaurants,Min Distance from Zip-Indian,Avg Distance from Zip-Indian,Percent of Indian Restaurants
0,4,"Elk Grove Village, IL",42.006304,-88.005855,60007,42.0076,-87.9931,"Elk Grove Village, US",1066.406871,7230.627861,10.59,33820,75743,56,947,2008.0,0.0,0.0,0.0,0.00
3,4,"Elk Grove Village, IL",42.006304,-88.005855,60143,41.9720,-88.0202,"Itasca, US",3991.366735,9840.646216,7.44,10360,88031,31,568,1754.0,0.0,0.0,0.0,0.00
4,4,"Elk Grove Village, IL",42.006304,-88.005855,60157,41.9814,-88.0512,"Medinah, US",4665.666767,8522.973014,5.05,2380,77536,41,560,2196.0,1.0,2178.0,2178.0,2.44
8,0,"Hoffman Estates, IL",42.065890,-88.121643,60195,42.0764,-88.1093,"Schaumburg, US",1551.207998,5450.085925,38.03,4769,83358,26,1280,2279.0,0.0,0.0,0.0,0.00
9,3,"Palatine, IL",42.119485,-88.041359,60004,42.1120,-87.9792,"Arlington Heights, US",5207.375760,8272.669875,8.35,50582,83431,34,53,1071.0,0.0,0.0,0.0,0.00
10,2,"Palatine, IL",42.119485,-88.041359,60010,42.1614,-88.1383,"Barrington, US",9268.193450,13681.636887,10.34,44095,133572,41,97,948.0,0.0,0.0,0.0,0.00
11,2,"Palatine, IL",42.119485,-88.041359,60047,42.2165,-88.0769,"Lake Zurich, US",11169.116825,17753.694608,8.55,41669,125778,11,2099,2366.0,0.0,0.0,0.0,0.00
12,0,"Palatine, IL",42.119485,-88.041359,60067,42.1139,-88.0429,"Palatine, US",633.311587,6213.475959,14.38,38585,90474,49,299,1362.0,0.0,0.0,0.0,0.00
13,3,"Palatine, IL",42.119485,-88.041359,60074,42.1458,-88.0230,"Palatine, US",3293.585446,9961.831594,12.97,38985,72946,68,413,1638.0,2.0,744.0,904.0,2.94
14,3,"Palatine, IL",42.119485,-88.041359,60089,42.1598,-87.9644,"Buffalo Grove, US",7780.084556,13253.194850,17.81,41533,101576,77,286,1810.0,2.0,362.0,1047.0,2.60


In [40]:
map_schaumburg = folium.Map(location=[lat_, lng_], zoom_start=11)
for row in suburbAddress:
    latilongi = [row[1],row[2]]
    popup = row[0]
    folium.Marker(latilongi, popup=popup).add_to(map_schaumburg)
#HeatMap(dataZipCodes).add_to(map_schaumburg)
for lat, lon in zip(zipRestaurants1['Latitude'], zipRestaurants1['Longitude']):
    color = 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_schaumburg)
for lat, lon in kmeans.cluster_centers_:
    folium.Circle([lat, lon], radius=3000, color='green', fill=False).add_to(map_schaumburg)
map_schaumburg

Let's get the addresses of the locations within zip codes where possibly Indian restaurants can be opened.

In [41]:
print("Address of locations around which Indian restaurants can be opened")
print('------------------------------------------------------------------')
for lat, lon in kmeans.cluster_centers_:
    location = geolocator1.reverse([lat,lon])
    print(location.address.replace(', USA', ''))

Address of locations around which Indian restaurants can be opened
------------------------------------------------------------------
2684, Mill Creek Lane, Rolling Meadows, Cook County, Illinois, 60008
448, East Lake Street, Streamwood, Cook County, Illinois, 60107
MB Financial Bank, Rand Road, Lake Zurich, Lake County, Illinois, 60047
2460, North Douglas Avenue, Arlington Heights, Cook County, Illinois, 60004
Terrace Drive, Medinah, DuPage County, Illinois, 60157


This concludes our analysis. We have identified five addresses in the vicinity of Schaumburg where percentage of Indian restaurants is less than 3%, Asian population is at least 5% and Household median income is greater than 70K. 

## Results and Discussion <a name="results"></a>

Our Analysis shows there are 30 unique zip codes in suburbs in the vicinity of Schaumburg. These zip codes have close to 1400 restaurants. Of these there are only 32 Indian restaurants, which constitute 2.26% of all restaurants	. Average number of restaurants in each zip code is around 47. Average distance of restaurants from center of zip code is 571m.


Of the 30 zip codes only 11 zip codes have Indian restaurants and average distance of these restaurants from center of zip code is 1368m.


Highest number of restaurants were detected in all directions from Schaumburg center except in North West. Indian restaurants are mostly concentrated in a triangle of Schaumburg, Hoffman Estates and Streamwood.
We also looked the demographics of the zip codes to get the total population, percentage of Asian Population and Median Household Income. 


We then narrowed down the number of zip codes to where percentage of Indian restaurants is less than 3%, Asian population is at least 5% and Median Household income is greater than 70K. This identified 18 target zip codes. We than clustered these zip codes to identify centers of points of interest. We also get the address of these centers by reverse geocoding.


Result of all this analysis is 5 zones which can be used as a starting point for finding the best location for opening an Indian restaurant. However, this is only an initial analysis and should be further analyzed by looking at other data e.g., crime rate, age grouping, education, commercial activity, real estate, fun areas etc. of the zip codes. Additional analysis might lead to different conclusions.


## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify areas in vicinity of Schaumburg with low number of Indian restaurants to help "Pardes India Inc" in narrowing down the search for optimal location for a new Indian restaurant. By calculating restaurant density distribution from Foursquare data we have first identified zip codes that warrent further analysis. Clustering of these zip codes was performed to create major zones of interest and addresses of those zone centers were created to be used as starting points for final exploration.

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.